In [ ]:
########## This code is used for the analysis of river surface temperature profiles in CONUS 2013-2024 ##########

In [ ]:
# Import Libraries
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from shapely.geometry import Point, Polygon
import glob
import os
import numpy as np
import seaborn as sns
import scipy.stats as stats
from scipy.stats import linregress
import pymannkendall as mk
import math

In [ ]:
#############################################
########### Pull in the initial datasets ############
############################################

In [ ]:
###  Pull in the Dam File ###
Dams = gpd.read_file(r"F:\Insert_File_Path_of_Shapefile_with_Dam_Locations.shp") # Update this file path
## This Shapefile ^^ has all the dams used to pull temperatures in it with infromation from HILARRI matched to it (completed in ArcGIS) ##
Dams

In [ ]:
### Pull in the Temperature Data -- Up/Ds FIltering ###
# Set up the location of the data
FilePath = r"F:\Insert_File_Path_for_Folder_of_Snapped_Temperatures_for_Each_Dam" ## Temperatures for each dam

# Get the List of Monthly Data
CSVFiles = glob.glob(os.path.join(FilePath, "*Avg_Img_Temps.csv")) 

# Loop through the files for each dam and make one dataframe
All_Mo_Avg = pd.DataFrame()
for i in range(len(CSVFiles)):
    try:
        x = pd.read_csv(CSVFiles[i], engine='python')
        All_Mo_Avg = pd.concat([All_Mo_Avg,x],axis=0)
    except pd.errors.EmptyDataError:
        print(CSVFiles[i], " is empty and has been skipped.") # In case some of the CSV files are empty
# Preview the data
All_Mo_Avg

In [ ]:
## Looking at just the wide points, and non dam points
All_Mo_Avg_Subset = All_Mo_Avg[(All_Mo_Avg['Avg_RWC_Wid'] >= 100) & (All_Mo_Avg['Up_Ds']!= 'Dam')]
All_Mo_Avg_Subset

In [ ]:
#### Clean up the Data ####
# Get Dam Info on the nodes/temperature Values  and clean up the dataset ##
HydroPower = pd.merge(All_Mo_Avg_Subset, Dams[['grod_id', 'dataset']], left_on = 'Assgn_dam', right_on = 'grod_id' , how='left').drop(columns = [ 'grod_id'])

# Remove Unwanted Columns
HydroDams_Clean = HydroPower.drop(columns=['Unnamed: 0'])

# Make an easier code for hydropower variables -- Updating the dictionary for simplicity
damtype_dict = {'Hydropower dam associated with power plant; no reservoir': 'HDNR', 'Hydropower dam associated with reservoir and power plant': 'HDR', 'Hydropower dam only; no reservoir or power plant': 'HDNR','Hydropower dam associated with reservoir; no power plant':'HDR', None: 'NoHydro'} # dictionary
HydroDams_Clean['HydroCode'] = HydroDams_Clean['dataset'].map(damtype_dict)  # apply dictionary

# Group datata into distance bins
bins = np.arange(-100, 100, 2).tolist() # create 2km bins
HydroDams_Clean['Bins'] = pd.cut(HydroDams_Clean['Dam_Dist_km'], bins) # apply bins

# Create a Code to easily group based on Up/Ds and Lake Flag
HydroDams_Clean['Up_Ds_Grp']  = np.nan
HydroDams_Clean['Up_Ds_Grp'] = np.where((HydroDams_Clean['Up_Ds'] == 'Downstream') & (HydroDams_Clean['lakeflag'] == 0) , 'Downstream River', HydroDams_Clean['Up_Ds_Grp'])
HydroDams_Clean['Up_Ds_Grp'] = np.where((HydroDams_Clean['Up_Ds'] == 'Downstream') & (HydroDams_Clean['lakeflag'] > 0) , 'Downstream Reservoir', HydroDams_Clean['Up_Ds_Grp'])
HydroDams_Clean['Up_Ds_Grp'] = np.where((HydroDams_Clean['Up_Ds'] == 'Upstream') & (HydroDams_Clean['lakeflag'] == 0) , 'Upstream River', HydroDams_Clean['Up_Ds_Grp'])
HydroDams_Clean['Up_Ds_Grp'] = np.where((HydroDams_Clean['Up_Ds'] == 'Upstream') & (HydroDams_Clean['lakeflag'] > 0) , 'Upstream Reservoir', HydroDams_Clean['Up_Ds_Grp'])

# Preview Data
HydroDams_Clean

In [ ]:
###########################################################
############## FILTER TO UNIQUE, USABLE PROFILES ##############
###########################################################

In [ ]:
## Get Profiles -- Where there are river nodes upstream ##
UpRiver= HydroDams_Clean[(HydroDams_Clean["Up_Ds_Grp"] == "Upstream River")]
UpRiver_grp = UpRiver.groupby(['Month','Day','Year','Assgn_dam']).agg({'Join_Node': ['count']})
UpRiver_grp.columns = ["NodeCount"]
UpRiver_grp = UpRiver_grp.reset_index()

## Identify Profiles with 5 Upstream River Nodes
DamTemps_RivUp = UpRiver_grp[UpRiver_grp["NodeCount"]>= 5]

## Get Profiles -- Where there are river nodes directly downstream from the dam ##
DsRiver= HydroDams_Clean[(HydroDams_Clean["Up_Ds_Grp"] == "Downstream River") & (HydroDams_Clean["Dam_Dist_km"] <= 20)]
DsRiver_grp = DsRiver.groupby(['Month','Day','Year','Assgn_dam']).agg({'Join_Node': ['count']})
DsRiver_grp.columns = ["NodeCount"]
DsRiver_grp = DsRiver_grp.reset_index()

## Identify Profiles with 5 Downstream River Nodes
DamTemps_RivDs = DsRiver_grp[DsRiver_grp["NodeCount"]>= 5]

#### NUMBER OF PROFILES ####
UpAnDown = pd.merge(DamTemps_RivDs, DamTemps_RivUp, on = ['Assgn_dam', "Month", "Day", "Year"] , how='inner')
UpAnDown

In [ ]:
## Save the unique profiles with enough points to a CSV
UpAnDown.to_csv(r"F:\Insert_File_Output_Path\List_of_Profiles.csv")

In [ ]:
####################################################################
############ Identify Profiles with Significant Up/DS Differences ##############
####################################################################

In [ ]:
## Get Profile Points (filtering only 20km DS)##
DamTemps_Riv20 = HydroDams_Clean[(HydroDams_Clean["Dam_Dist_km"] <= 20)]

## Calculate Significance (Up River/20km)##
col_names =  ['Assgn_dam','Month','Day','Year', 'KW_Pval']
KW_Prof_Results_Riv20  = pd.DataFrame(columns = col_names)
Unique_Profiles_Riv20 =  UpAnDown[["Assgn_dam", "Month","Day", "Year"]]

for i in range(len(Unique_Profiles_Riv20)):  # For selecting -- .iloc[i, 0] ## i for row , Dam Number (0), Month (1), Day (2), Year(3)
    # Set up the subsets for comparison -- All
    KruskalTest_1 = DamTemps_Riv20[(DamTemps_Riv20['Assgn_dam'] == Unique_Profiles_Riv20.iloc[i, 0]) & (DamTemps_Riv20['Month'] == Unique_Profiles_Riv20.iloc[i, 1]) & (DamTemps_Riv20['Day'] == Unique_Profiles_Riv20.iloc[i, 2]) & (DamTemps_Riv20['Year'] == Unique_Profiles_Riv20.iloc[i, 3]) & (DamTemps_Riv20['Up_Ds_Grp'] == "Downstream River")].Avg_Temp.tolist() # Downstream
    KruskalTest_2 = DamTemps_Riv20[(DamTemps_Riv20['Assgn_dam'] == Unique_Profiles_Riv20.iloc[i, 0]) & (DamTemps_Riv20['Month'] == Unique_Profiles_Riv20.iloc[i, 1]) & (DamTemps_Riv20['Day'] == Unique_Profiles_Riv20.iloc[i, 2]) & (DamTemps_Riv20['Year'] == Unique_Profiles_Riv20.iloc[i, 3]) & (DamTemps_Riv20['Up_Ds_Grp'] == "Upstream River")].Avg_Temp.tolist() # Upstream    
     
    # Run the test -- River vs DS
    h_statistic, p_value = stats.kruskal(KruskalTest_2,KruskalTest_1)

    # Get Dictionary
    dictionary = {'Assgn_dam': Unique_Profiles_Riv20.iloc[i, 0], 'Month': Unique_Profiles_Riv20.iloc[i, 1], 'Day': Unique_Profiles_Riv20.iloc[i, 2], 'Year': Unique_Profiles_Riv20.iloc[i, 3], 'KW_Pval': [p_value]}

    df_dictionary = pd.DataFrame.from_dict(dictionary)
    
    # Add to DF
    output = pd.concat([KW_Prof_Results_Riv20, df_dictionary], ignore_index=True)
    
    KW_Prof_Results_Riv20 = output

In [ ]:
print("Number of Profiles with >=5 Riv Nodes Upstream and Downstream: ", len(Unique_Profiles_Riv20))

In [ ]:
# Define Significant profiles (Riv Up/20km)
KW_Prof_Results_Riv20['KW_Sig_10'] = np.where((KW_Prof_Results_Riv20['KW_Pval'] <= 0.1), 'Significant', 'Not Significant')
KW_Prof_Results_Riv20['KW_Sig_05'] = np.where((KW_Prof_Results_Riv20['KW_Pval'] <= 0.05), 'Significant', 'Not Significant')
KW_Prof_Results_Riv20['KW_Sig_01'] = np.where((KW_Prof_Results_Riv20['KW_Pval'] <= 0.01), 'Significant', 'Not Significant')
KW_Prof_Results_Riv20['KW_Sig_001'] = np.where((KW_Prof_Results_Riv20['KW_Pval'] <= 0.001), 'Significant', 'Not Significant')
KW_Prof_Results_Riv20

In [ ]:
# Get the Significant Profiles within 20km
Signif_Riv20_Profiles = KW_Prof_Results_Riv20[KW_Prof_Results_Riv20["KW_Sig_05"] == "Significant"]
print("Number of Profiles with Signficant differences Up/Ds: ", len(Signif_Riv20_Profiles))

### Percent Significant ### 
print("Percent Significant Profiles: ", round((len(Signif_Riv20_Profiles)/len(Unique_Profiles_Riv20))*100), "%")

# Get Relevant nodes
Signif_Riv20_Profile_Nodes = pd.merge(DamTemps_Riv20, Signif_Riv20_Profiles, on=['Month', 'Day','Year','Assgn_dam'], how='inner')

In [ ]:
# Get the Not Significant Profiles within 20km
NS_Riv20_Profiles = KW_Prof_Results_Riv20[KW_Prof_Results_Riv20["KW_Sig_05"] == "Not Significant"]

# Get Relevant nodes
NS_Riv20_Profile_Nodes = pd.merge(DamTemps_Riv20, NS_Riv20_Profiles, on=['Month', 'Day','Year','Assgn_dam'], how='inner')

In [ ]:
## Save the Results ## 
KW_Prof_Results_Riv20.to_csv(r"F:\Insert_File_Output_for_Significance_Results.csv") # Update this file path
Signif_Riv20_Profile_Nodes.to_csv(r"F:\Insert_File_Output_for_Significant_nodes.csv") # Update this file path
NS_Riv20_Profile_Nodes.to_csv(r"F:\Insert_File_Output_for_Nonsignificant_nodes.csv") # Update this file path

In [ ]:
###########################################################
########### Get Average Temperature Differences ################
###########################################################

In [ ]:
## Get the Avg Diff Value for the profiles  ##
# Get the Profiles and Nodes -- Copy
SignificantProfiles_Riv = Signif_Riv20_Profiles
SignificantProfiles_Riv_Nodes = Signif_Riv20_Profile_Nodes

# Of The Significant Profiles -- Avg Diff Up Vs Ds 
Upstream_Riv = SignificantProfiles_Riv_Nodes[(SignificantProfiles_Riv_Nodes['Up_Ds_Grp'] == "Upstream River")]
Upstream_Avgs_Riv = Upstream_Riv.groupby(['Month','Day','Year','Assgn_dam']).agg({'Avg_Temp': ['mean']})
Upstream_Avgs_Riv.columns = ['Up_Avg']
Upstream_Avgs_Riv = Upstream_Avgs_Riv.reset_index()

# Downstream_Riv = SignificantProfiles_Riv_Nodes[(SignificantProfiles_Riv_Nodes['Up_Ds'] == 'Downstream')]
Downstream_Riv = SignificantProfiles_Riv_Nodes[(SignificantProfiles_Riv_Nodes['Up_Ds_Grp'] == 'Downstream River')]
Downstream_Avgs_Riv = Downstream_Riv.groupby(['Month','Day','Year','Assgn_dam']).agg({'Avg_Temp': ['mean']})
Downstream_Avgs_Riv.columns = ['Ds_Avg']
Downstream_Avgs_Riv = Downstream_Avgs_Riv.reset_index()

Profile_Avgs_Riv = pd.merge(Upstream_Avgs_Riv, Downstream_Avgs_Riv, on=['Month','Day', 'Year','Assgn_dam'], how='outer' )

Profile_Avgs_Riv['Temp_Diff'] = Profile_Avgs_Riv['Ds_Avg'] - Profile_Avgs_Riv['Up_Avg'] 
Profile_Avgs_Riv['Abs_Temp_Diff'] = abs(Profile_Avgs_Riv['Ds_Avg'] - Profile_Avgs_Riv['Up_Avg'])

## Save the Results ## 
Profile_Avgs_Riv.to_csv(r"F:\Insert_File_Path_for_Signifcant_Profile_Averages.csv") # Update this file path

In [ ]:
## Warmer Profiles 
print("Number of Warmer Profiles: " + str(len(Profile_Avgs_Riv[Profile_Avgs_Riv['Temp_Diff'] > 0])))
print("Percent Warmer: " + str(round((len(Profile_Avgs_Riv[Profile_Avgs_Riv['Temp_Diff'] > 0]) / len(Profile_Avgs_Riv)*100)))+ "%")


In [ ]:
## Cooler Profiles
print("Number of Cooler Profiles: " + str(len(Profile_Avgs_Riv[Profile_Avgs_Riv['Temp_Diff'] < 0])))
print("Percent Cooler: " + str(round((len(Profile_Avgs_Riv[Profile_Avgs_Riv['Temp_Diff'] < 0]) / len(Profile_Avgs_Riv)*100)))+ "%")

In [ ]:
###############################################################

In [ ]:
### Look at the differences seasonally ## 
Seasonal_Diffs = Profile_Avgs_Riv[:]

# Assign seasons according to the months
Seasonal_Diffs["Season"] = np.nan
Seasonal_Diffs["Season"] = np.where(Seasonal_Diffs["Month"].isin([12, 1, 2]), "Winter", Seasonal_Diffs["Season"])
Seasonal_Diffs["Season"] = np.where(Seasonal_Diffs["Month"].isin([3, 4, 5]), "Spring", Seasonal_Diffs["Season"])
Seasonal_Diffs["Season"] = np.where(Seasonal_Diffs["Month"].isin([6,7,8]), "Summer", Seasonal_Diffs["Season"])
Seasonal_Diffs["Season"] = np.where(Seasonal_Diffs["Month"].isin([9, 10, 11]), "Fall", Seasonal_Diffs["Season"])

# Get the Direction of the temperature change
Seasonal_Diffs["Diff_Direction"] = np.nan
Seasonal_Diffs["Diff_Direction"] = np.where((Seasonal_Diffs["Temp_Diff"] < 0) , 'Cooler', Seasonal_Diffs["Diff_Direction"])
Seasonal_Diffs["Diff_Direction"] = np.where((Seasonal_Diffs["Temp_Diff"] > 0) , 'Warmer', Seasonal_Diffs["Diff_Direction"])

## Get the profile stats by season 

# create a range function
def calculate_range(x):
    return x.max() - x.min()

## Get Group Info
Profile_Stats_Season = Seasonal_Diffs.groupby(["Season","Diff_Direction"]).agg({'Temp_Diff':['count','mean', 'std', calculate_range]})
Profile_Stats_Season.columns = ['Profile_Count','Avg_Diff', 'STDV', 'Range']
Profile_Stats_Season = Profile_Stats_Season.reset_index()

## Save File 
Profile_Stats_Season.to_csv(r"F:\Insert_File_Path_for_Profile_Stats_by_Season.csv") # Update this file path

# Preview --- Table S3
Profile_Stats_Season

In [ ]:
###########################################################
############## Calculate the Profile Anomalies ##################
###########################################################

In [ ]:
## By Dam: Annual Anomalies ## -- Figure 4A
# Get List of Dams that have at least one profile with 5 up and 5 down nodes
Dams_with_Nodes = UpAnDown["Assgn_dam"].drop_duplicates().to_list()
Anom_Subset = HydroDams_Clean[HydroDams_Clean['Assgn_dam'].isin(Dams_with_Nodes)]

# Get the average temperature for each month and dam
Dam_Avg_All= Anom_Subset.groupby(['Assgn_dam','Year']).agg({'Avg_Temp': ['mean']})
Dam_Avg_All.columns = ['Prof_Avg']
Dam_Avg_All = Dam_Avg_All.reset_index()

# Join the profile averages back to each node
Dam_Anom_join_all = pd.merge(Anom_Subset, Dam_Avg_All, on=['Assgn_dam','Year'], how = 'left')

# Calcuate the difference between the node value and the mean temperature for the given profile 
Dam_Anom_join_all['Temp_Anom'] = Dam_Anom_join_all['Avg_Temp'] - Dam_Anom_join_all['Prof_Avg']  ## Postive Values = warmer than the mean, Negative = colder than the mean 

# Remove Points far downstream
Dam_Anom_join_all_sub = Dam_Anom_join_all[Dam_Anom_join_all['Dam_Dist_km'] <= 60]

# Get the Average Anomalies for Each Dam (All)
Avg_Anom_byDam_all = Dam_Anom_join_all_sub.groupby(["Up_Ds_Grp", "Month","Day","Year","Assgn_dam"]).agg({'Temp_Anom':['mean','count']})
Avg_Anom_byDam_all.columns = ['Anom_Avg', 'NPoints']
Avg_Anom_byDam_all = Avg_Anom_byDam_all.reset_index()


### Plot the Anomalies ###
# Set the order of groups for boxplots
grouporder = ['Upstream River', 'Upstream Reservoir', 'Downstream River']

# Plot
plt.figure(figsize=(6,4))
ax = sns.boxplot(x="Month", y="Anom_Avg",
                 hue = "Up_Ds_Grp", hue_order=grouporder,
                 palette="winter_r",
                 linewidth=0.75,
                 showfliers = False, # Comment this out to see outliers
                 data =Avg_Anom_byDam_all, 
                 zorder = 3)
ax.set_xlabel( "Month", fontsize=12) 
ax.set_ylabel( "Average Temperature Anomaly (°C)", fontsize=12) 
ax.tick_params(labelsize=12)
ax.yaxis.set_minor_locator(MultipleLocator(5))
sns.move_legend(ax, title='', fontsize='8', loc ="upper left")
plt.savefig(r"F:\Insert_File_Path_for_Anomalies_AtDam_Annual.png", bbox_inches="tight", pad_inches=0.25, dpi=1200) # Update this file path

In [ ]:
## Get Variance and STDV ##
# Get Values for Each Month
Avg_Anom_STDV_Dam = Avg_Anom_byDam_all.groupby(["Month", "Up_Ds_Grp"]).agg({'Anom_Avg':['std','var']})
Avg_Anom_STDV_Dam.columns = ['Anom_STDV', 'Anom_Var']
Avg_Anom_STDV_Dam = Avg_Anom_STDV_Dam.reset_index()

In [ ]:
#############################################

In [ ]:
## By Profile: Spatial Anomalies ## -- Figure 4B
# Get List of Dams that have at least one profile with 5 up and 5 down nodes
Dams_with_Nodes = UpAnDown["Assgn_dam"].drop_duplicates().to_list()
Anom_Subset = HydroDams_Clean[HydroDams_Clean['Assgn_dam'].isin(Dams_with_Nodes)]

# Get the average temperature for each profile
Anom_grp = Anom_Subset.groupby(['Month','Day','Year','Assgn_dam']).agg({'Avg_Temp': ['mean']})
Anom_grp.columns = ['Prof_Avg']
Anom_grp = Anom_grp.reset_index()

## Save the Spatial Average Profile Temperature
Anom_grp.to_csv(r"F:\Insert_File_Path_for_All_Entire_Profile_Averages.csv") # Update this file path

# Join the profile averages back to each node
Anom_join = pd.merge(Anom_Subset, Anom_grp, on=['Assgn_dam','Month','Day', 'Year'], how = 'left')

# Calcuate the difference between the node value and the mean temperature for the given profile 
Anom_join['Temp_Anom'] = Anom_join['Avg_Temp'] - Anom_join['Prof_Avg']  ## Postive Values = warmer than the mean, Negative = colder than the mean 

# Remove Points far downstream
Anom_join_sub = Anom_join[Anom_join['Dam_Dist_km'] <= 60]

# Get the Average Anomalies for Each Profile and Group
Avg_Anom_byDam = Anom_join_sub.groupby(["Up_Ds_Grp", "Month","Day","Year","Assgn_dam"]).agg({'Temp_Anom':['mean','count']})
Avg_Anom_byDam.columns = ['Anom_Avg', 'NPoints']
Avg_Anom_byDam = Avg_Anom_byDam.reset_index()

### Plot the Anomalies ### 
# Set the order of groups for boxplots
grouporder = ['Upstream River', 'Upstream Reservoir', 'Downstream River']

# Plot
plt.figure(figsize=(6,4))
ax = sns.boxplot(x="Month", y="Anom_Avg",
                 hue = "Up_Ds_Grp", hue_order=grouporder,
                 palette="winter_r",
                 linewidth=0.75,
                 showfliers = False, # Comment this out to see outliers
                 data =Avg_Anom_byDam, 
                 zorder = 3)
ax.set_xlabel( "Month", fontsize=12) 
ax.set_ylabel( "Average Temperature Anomaly (°C)", fontsize=12) 
ax.set_ylim(-3,3)
plt.locator_params(axis='y', nbins= 5)
ax.tick_params(labelsize=12)
ax.yaxis.set_minor_locator(MultipleLocator(1))

# sns.move_legend(ax, title='', fontsize='8', loc ="upper right")
ax.get_legend().remove()

plt.savefig(r"F:\Insert_File_Path_Profile_Anomalies.png", bbox_inches="tight", pad_inches=0.25, dpi=1200) # Update this file path

In [ ]:
### Evaluate if the anomlies are sigfnificantly different by months ### 

# List of Months
Months = [1,2,3,4,5,6,7,8,9,10,11,12]

col_names =  ['Month', 'All_KW_Pval', 'Riv_KW_Pval', 'RR_KW_Pval', 'Res_KW_Pval']
KW_Seas_Results  = pd.DataFrame(columns = col_names)

for i in Months:
    # Set up the subsets for comparison -- All
    KruskalTest_1 = Avg_Anom_byDam[(Avg_Anom_byDam['Month'] == i) & (Avg_Anom_byDam['Up_Ds_Grp'] == "Downstream River")].Anom_Avg.tolist() # DS River
    KruskalTest_2 = Avg_Anom_byDam[(Avg_Anom_byDam['Month'] == i) & (Avg_Anom_byDam['Up_Ds_Grp'] == "Upstream River")].Anom_Avg.tolist() # UP River
    KruskalTest_3 = Avg_Anom_byDam[(Avg_Anom_byDam['Month'] == i) & (Avg_Anom_byDam['Up_Ds_Grp'] == "Upstream Reservoir")].Anom_Avg.tolist()  # UP Resrv
    
    # Run the test -- All 
    h_statistic, p_value = stats.kruskal(KruskalTest_1, KruskalTest_2, KruskalTest_3)

    # Run the test -- Up River vs Ds River
    h_statistic_riv, p_value_riv = stats.kruskal(KruskalTest_2,KruskalTest_1)

    # Run the test -- River vs Resv (UP)
    h_statistic_rr, p_value_rr = stats.kruskal(KruskalTest_2, KruskalTest_3)

    # Run the test -- Up Resv vs Ds River
    h_statistic_res, p_value_res = stats.kruskal(KruskalTest_3, KruskalTest_1)
    

    # Get Dictionary
    dictionary = {'Month': i , 'All_KW_Pval': [p_value], 'Riv_KW_Pval': [p_value_riv], 'RR_KW_Pval': [p_value_rr], 'Res_KW_Pval': [p_value_res]}

    df_dictionary = pd.DataFrame.from_dict(dictionary)
    
    # Add to DF
    output = pd.concat([KW_Seas_Results, df_dictionary], ignore_index=True)
    
    KW_Seas_Results = output

## Save and preview the significance by month
KW_Seas_Results['All_KW_Pval_Sig'] = np.where((KW_Seas_Results['All_KW_Pval'] <= 0.05), 'Significant', 'Not Significant') # All Categories
KW_Seas_Results['Riv_KW_Pval_Sig'] = np.where((KW_Seas_Results['Riv_KW_Pval'] <= 0.05), 'Significant', 'Not Significant') # Up River to DS River
KW_Seas_Results['RR_KW_Pval_Sig'] = np.where((KW_Seas_Results['RR_KW_Pval'] <= 0.05), 'Significant', 'Not Significant') # Up River to Up Reserv
KW_Seas_Results['Res_KW_Pval_Sig'] = np.where((KW_Seas_Results['Res_KW_Pval'] <= 0.05), 'Significant', 'Not Significant') # Up Reserv to DS River
KW_Seas_Results

In [ ]:
## Get Variance and STDV ##
# Get Values for Each Month
Avg_Anom_STDV_Prof = Avg_Anom_byDam.groupby(["Month", "Up_Ds_Grp"]).agg({'Anom_Avg':['std','var']})
Avg_Anom_STDV_Prof.columns = ['Anom_STDV', 'Anom_Var']
Avg_Anom_STDV_Prof = Avg_Anom_STDV_Prof.reset_index()

In [ ]:
###########################################################
########## Get Downstream Differences by Distance ###############
###########################################################

In [ ]:
## Get a Copy of All Temps for the Relevant Profiles
Profile_Temp_Points = pd.merge(HydroDams_Clean, UpAnDown, on=['Assgn_dam','Month','Day', 'Year'], how = 'inner')
Profile_Temp_Points = Profile_Temp_Points[['Join_Node', 'Assgn_dam', 'Month', 'Day', 'Year', 'Avg_Temp', 'Avg_RWC_Wid', 'x', 'y', 'reach_id', 
                                          'lakeflag', 'Dam_Flag', 'Up_Ds', 'Up_Ds_Grp', 'Dam_Dist', 'Dam_Dist_km', 'HydroCode']] # clean up columns

# List of Profiles with Up and DS Averages
# Get Up Avg 
Upstream_Points = Profile_Temp_Points[(Profile_Temp_Points['Up_Ds_Grp'] == "Upstream River")]
Upstream_Avg = Upstream_Points.groupby(['Month','Day','Year','Assgn_dam']).agg({'Avg_Temp': ['mean']})
Upstream_Avg.columns = ['Up_Avg']
Upstream_Avg = Upstream_Avg.reset_index()

# Get DS 20km Avg
Downstream_Riv = Profile_Temp_Points[(Profile_Temp_Points['Up_Ds_Grp'] == 'Downstream River') & (Profile_Temp_Points['Dam_Dist_km'] <= 20) ]
Downstream_Avgs_Riv = Downstream_Riv.groupby(['Month','Day','Year','Assgn_dam']).agg({'Avg_Temp': ['mean']})
Downstream_Avgs_Riv.columns = ['Ds_Avg_20']
Downstream_Avgs_Riv = Downstream_Avgs_Riv.reset_index()

Profile_Avgs_UPDS_Riv = pd.merge(Upstream_Avgs_Riv, Downstream_Avgs_Riv, on=['Month','Day', 'Year','Assgn_dam'], how='outer' )

Profile_Avgs_UPDS_Riv['Temp_Diff_20'] = Profile_Avgs_UPDS_Riv['Ds_Avg_20'] - Profile_Avgs_UPDS_Riv['Up_Avg'] 
Profile_Avgs_UPDS_Riv['Abs_Temp_Diff_20'] = abs(Profile_Avgs_UPDS_Riv['Ds_Avg_20'] - Profile_Avgs_UPDS_Riv['Up_Avg'])
Profile_Avgs_UPDS_Riv['Temp_Direction'] = np.where(Profile_Avgs_UPDS_Riv['Temp_Diff_20']>0, "Warmer", "Cooler")

## Combine Averages back to Points ## 
Profile_Absolute_Diff = pd.merge(Profile_Temp_Points,Profile_Avgs_UPDS_Riv, on=['Assgn_dam','Month','Day', 'Year'], how='left' )

## Get Absolute Differences ##
Profile_Absolute_Diff['Diff_UP_Node'] = Profile_Absolute_Diff['Avg_Temp'] - Profile_Absolute_Diff['Up_Avg']
Profile_Absolute_Diff['Abs_Diff_UP_Node'] = abs(Profile_Absolute_Diff['Avg_Temp'] - Profile_Absolute_Diff['Up_Avg'])

# Get the Signif Attached 
Signif_Flag = KW_Prof_Results_Riv20[['Assgn_dam','Month','Day', 'Year', 'KW_Sig_05']]

Profile_Differences = pd.merge(Profile_Absolute_Diff,Signif_Flag, on=['Assgn_dam','Month','Day', 'Year'], how='left' )

## Save the File ## 
Profile_Differences.to_csv(r"F:\Insert_File_Path_Profile_Points_Diffs.csv") # Update this file path

In [ ]:
##### Prepare to plot it ##### -- Figure 3D
Plot_Diff = Profile_Differences[:]

# Filter to less than 60 km downstream
Plot_Diff = Plot_Diff[Plot_Diff["Dam_Dist_km"] <= 60]

# Filter to just river nodes 
Plot_Diff = Plot_Diff[Plot_Diff["Up_Ds_Grp"] == "Downstream River"]

# Group by season, distance, and increasing or decreasing
Plot_Diff['Dam_Dist_km_c'] = Plot_Diff['Dam_Dist_km'].apply(math.ceil)

# Assign seasons according to the months
Plot_Diff["Season"] = np.nan
Plot_Diff["Season"] = np.where(Plot_Diff["Month"].isin([12, 1, 2]), "Winter", Plot_Diff["Season"])
Plot_Diff["Season"] = np.where(Plot_Diff["Month"].isin([3, 4, 5]), "Spring", Plot_Diff["Season"])
Plot_Diff["Season"] = np.where(Plot_Diff["Month"].isin([6,7,8]), "Summer", Plot_Diff["Season"])
Plot_Diff["Season"] = np.where(Plot_Diff["Month"].isin([9, 10, 11]), "Fall", Plot_Diff["Season"])

### Use only SIgnificant Differences ###
Plot_Diff_Signif =Plot_Diff[(Plot_Diff["KW_Sig_05"] == "Significant") & (Plot_Diff["Dam_Dist_km_c"] <= 20)]

# Group by season, distance, and increasing or decreasing
Plot_Diff_Signif['Dam_Dist_km_c'] = Plot_Diff_Signif['Dam_Dist_km'].apply(math.ceil)
Plot_Diff_Signif_Season = Plot_Diff_Signif.groupby(["Season", "Dam_Dist_km_c", "Temp_Direction"]).agg({'Diff_UP_Node':['mean']})
Plot_Diff_Signif_Season.columns=["Diff_UP_Node"]
Plot_Diff_Signif_Season = Plot_Diff_Signif_Season.reset_index()

Plot_Diff_Signif_Annual = Plot_Diff_Signif.groupby(["Dam_Dist_km_c", "Temp_Direction"]).agg({'Diff_UP_Node':['mean']})
Plot_Diff_Signif_Annual.columns=["Diff_UP_Node"]
Plot_Diff_Signif_Annual = Plot_Diff_Signif_Annual.reset_index()

##### Plot the Avg DS difference (1km) for first 20km of profiles with significant differences ###

pal = ["#031730",  "#F94994", "#006A5C","#EB9911"]

fig, ax=plt.subplots(figsize=(10,4))

ax =  sns.lineplot(x="Dam_Dist_km_c", y="Diff_UP_Node", data=Plot_Diff_Signif_Season[Plot_Diff_Signif_Season["Temp_Direction"] == "Warmer"],
            hue="Season", palette=sns.color_palette(pal), hue_order=["Winter", "Spring", "Summer", "Fall"], zorder=2)

sns.lineplot(x="Dam_Dist_km_c", y="Diff_UP_Node", data=Plot_Diff_Signif_Season[Plot_Diff_Signif_Season["Temp_Direction"] == "Cooler"],
            hue="Season", palette=sns.color_palette(pal), hue_order=["Winter", "Spring", "Summer", "Fall"], zorder=2, legend=False)

sns.lineplot(x="Dam_Dist_km_c", y="Diff_UP_Node", data=Plot_Diff_Signif_Annual[Plot_Diff_Signif_Annual["Temp_Direction"] == "Warmer"], color='#6F4E37', label="Annual", linestyle='dashed',  linewidth=2, zorder=2)
sns.lineplot(x="Dam_Dist_km_c", y="Diff_UP_Node", data=Plot_Diff_Signif_Annual[Plot_Diff_Signif_Annual["Temp_Direction"] == "Cooler"], color='#6F4E37',  linestyle='dashed',  linewidth=2, zorder=2, legend=False)

ax.set_xlim(1, 20)
ax.set_ylim(-2, 2)
ax.set_xticks([2,4,6,8,10,12,14,16,18,20])
ax.yaxis.set_minor_locator(MultipleLocator(0.25))
ax.tick_params(labelsize=12)
ax.axhline(0, linewidth = 0.8, color='black')

r = [-1.5, -1, -0.5, 0, 0.5, 1, 1.5]
for i in r:
    ax.axhline(i, linewidth=0.8, color='lightgrey', zorder=0)

for i in range(1, 21, 1):
    ax.axvline(i, linewidth=0.8, color='lightgrey', zorder=0)

ax.set_xlabel("Downstream Distance from dam (km)",  fontsize=14)
ax.set_ylabel("Temperature Difference ($^\circ$C)",  fontsize=14)
ax.legend(title=None, ncol=3, fontsize=12)

ax.spines['top'].set_color('black')
ax.spines['right'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_color('black')

plt.savefig(r"F:\Insert_File_Path_Avg_DS_Dist_20km.png", bbox_inches="tight", pad_inches=0.25, dpi=1200) # Update this file path

In [ ]:
## Look for Significant trends with DS Distance
Slope_P_Col = ['Season', 'Direction', 'Slope', 'Pval']
DS_Dist_Slope_P = pd.DataFrame(columns = Slope_P_Col)

# Get Unique Groupings
Groups = Plot_Diff_Signif_Season.groupby(['Season','Temp_Direction']).agg({'Dam_Dist_km_c': ['count']})
Groups.columns = ['Count']
Groups = Groups.reset_index()

for i in range(len(Groups)):
    Group_Filter = Plot_Diff_Signif_Season[(Plot_Diff_Signif_Season['Season'] == Groups.iloc[i,0]) & (Plot_Diff_Signif_Season['Temp_Direction'] == Groups.iloc[i,1])]

    Dist = Group_Filter['Dam_Dist_km_c'].tolist()
    Temp = Group_Filter['Diff_UP_Node'].tolist()

    # Get the regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(Dist, Temp)

    # Get Dictionary
    dictionary = {'Season': Groups.iloc[i, 0], 'Direction': Groups.iloc[i,1], 'Slope': [slope], 'Pval': [p_value]}
    
    df_dictionary = pd.DataFrame.from_dict(dictionary)
    
    # Add to DF
    output = pd.concat([DS_Dist_Slope_P, df_dictionary], ignore_index=True)
    
    DS_Dist_Slope_P = output

DS_Dist_Slope_P["Signif"] = np.where((DS_Dist_Slope_P['Pval'] <= 0.05), 'Significant', 'Not Significant')
DS_Dist_Slope_P

In [ ]:
## Check Significance of Annual Warm
Warm = Plot_Diff_Signif_Season[(Plot_Diff_Signif_Season['Temp_Direction'] == 'Warmer')]

Dist_w = Warm['Dam_Dist_km_c'].tolist()
Temp_w = Warm['Diff_UP_Node'].tolist()

# Get the regression
slope, intercept, r_value, p_value, std_err = stats.linregress(Dist_w, Temp_w)

print ("Slope: " + str(slope) + ", P Value: " + str(p_value))

In [ ]:
## Check Significance of Annual Cool
Cool = Plot_Diff_Signif_Season[(Plot_Diff_Signif_Season['Temp_Direction'] == 'Cooler')]

Dist_c = Cool['Dam_Dist_km_c'].tolist()
Temp_c = Cool['Diff_UP_Node'].tolist()

# Get the regression
slope, intercept, r_value, p_value, std_err = stats.linregress(Dist_c, Temp_c)

print ("Slope: " + str(slope) + ", P Value: " + str(p_value))

In [ ]:
######## Plot The Downstream  Anomalies -- Figure 4C ########
# Subset the data
Riv_Diff_Anom = Profile_Differences[Profile_Differences['Dam_Dist_km'] <= 60]

# Get the Average Anomalies
Avg_Anom_byUPRiv = Riv_Diff_Anom.groupby(["Up_Ds_Grp", "Month","Day","Year","Assgn_dam"]).agg({'Diff_UP_Node':['mean','count']})
Avg_Anom_byUPRiv.columns = ['Anom_Avg', 'NPoints']
Avg_Anom_byUPRiv = Avg_Anom_byUPRiv.reset_index()
Avg_Anom_byUPRiv


# Set the order of groups for boxplots
grouporder = ['Upstream River', 'Upstream Reservoir', 'Downstream River']

# Plot
plt.figure(figsize=(6,4))
ax = sns.boxplot(x="Month", y="Anom_Avg",
                 hue = "Up_Ds_Grp", hue_order=grouporder,
                 palette="winter_r",
                 linewidth=0.75,
                 showfliers = False, # Comment this out to see outliers
                 data =Avg_Anom_byUPRiv, 
                 zorder = 3)
ax.set_xlabel( "Month", fontsize=12)
ax.set_ylabel( "Average Temperature Anomaly (°C)", fontsize=12) 
ax.tick_params(labelsize=12)
ax.set_ylim(-7,7)
ax.yaxis.set_minor_locator(MultipleLocator(1))
ax.get_legend().remove()

plt.savefig(r"F:\Insert_File_Path_to_Anomalies_to_UPRiver.png", bbox_inches="tight", pad_inches=0.25, dpi=1200) # Update this file path

In [ ]:
###########################################
####### Create some base info for graphics #######
###########################################

In [ ]:
### Create a study area map ### -- Figure 1A
# Get List of Dams Used
Dams_of_Interest = Profile_Differences["Assgn_dam"].unique().tolist()
Dams_of_Interest.sort()

# Fliter the List 
BestDams = Dams[Dams["grod_id"].isin(Dams_of_Interest)]

## Set up Map ##
# State Boundaries
USA = gpd.read_file(r"F:\Insert_Path_to_USA_State_Outlines.shp") ## Shapefile of the USA to be filtered and mapped # Update this file path
USA=USA.to_crs(4326)

# Filter USA file to CONUS if needed
CONUS = USA[(~USA['NAME'].isin([ 'Alaska','American Samoa', 'District of Columbia', 'Fed States of Micronesia', 'Guam', 'Hawaii','Marshall Islands', 'Northern Mariana Islands','Palau', 'Puerto Rico', 'Virgin Islands', 'District of Columbia']))]

## Large Rivers
SWORD = gpd.read_file(r"F:\Insert_Path_to_NA_SWORD_reach_v16_gt100_map.shp") ## Combined SWORD centerlines > 100m, completed in ArcGIS # Update this file path
SWORD = SWORD.to_crs(4326)

## 87 Gages used in AA
GageSites_gdf =  gpd.read_file(r"F:\Insert_Path_to_USGS_Gages_AA_n87.shp")  # Update this file path
GageSites_gdf = GageSites_gdf.to_crs(4326)

from shapely.geometry import Point, LineString, Polygon
bufferforplot = SWORD.buffer(0.1) ## Buffered for visualization purposes only 

# Plot Data
fig, ax=plt.subplots(figsize=(14,10))
CONUS.plot(ax=ax,color='whitesmoke', edgecolor='dimgray', linewidth = 0.7, alpha = 0.75, zorder = 1)
bufferforplot.plot(ax=ax,color='#006c7a',  linewidth = 0.01, alpha = 0.9,  label = "Rivers (>100m)", zorder = 2)
BestDams.plot(ax=ax, marker='^', markersize = 50, color = "#fb5607", edgecolor = "#F98D5A",  linewidth = 0.5, label = "Dams", zorder=4)
GageSites_gdf.plot( ax = ax, marker = "o", markersize = 50, color = '#5603ad', edgecolor = "#8367c7", linewidth = 0.5,  label = "USGS Gages", zorder=3)
fig.legend(loc = "outside upper right")

fig.savefig(r"F:\Insert_Path_to_Study_Area_Map.png", bbox_inches="tight", pad_inches=0.25, dpi=1200) # Update this file path

In [ ]:
################################## GET PROFILE INFO ##################################

In [ ]:
### Prepare Number of Profiles by Season Graphic -- Figure 3A 
# Get the total number of profiles per month
NumberofProfiles = KW_Prof_Results_Riv20.groupby("Month").agg({'Month': ['count']})
NumberofProfiles.columns = ["Month_Count"]
NumberofProfiles = NumberofProfiles.reset_index()

# Assign the seasons
NumberofProfiles["Season"] = np.nan
NumberofProfiles["Season"] = np.where(NumberofProfiles['Month'].isin([12, 1, 2]), 'Winter', NumberofProfiles['Season'])
NumberofProfiles["Season"] = np.where(NumberofProfiles['Month'].isin([3, 4, 5]), 'Spring', NumberofProfiles['Season'])
NumberofProfiles["Season"] = np.where(NumberofProfiles['Month'].isin([6, 7, 8]), 'Summer', NumberofProfiles['Season'])
NumberofProfiles["Season"] = np.where(NumberofProfiles['Month'].isin([9, 10, 11]), 'Fall', NumberofProfiles['Season'])

## Plot the profile counts 
ColorPalette = ['#031730','#F94994', '#006A5C', '#EB9911']
HueOrder = ["Winter","Spring","Summer","Fall"]

fig, ax=plt.subplots(figsize=(6,4))
ax = sns.barplot(NumberofProfiles, x="Month", y="Month_Count", hue ="Season", hue_order = HueOrder, palette = ColorPalette)
ax.set_xlabel( "Month", fontsize=14) 
ax.set_ylabel( "Number of Profiles", fontsize=14) 
ax.set_title("Total Number of Profiles by Month 2013-2024", fontsize = 14)
ax.tick_params(labelsize=12)
ax.legend(title=None, ncol=2, fontsize=12)
ax.yaxis.set_minor_locator(MultipleLocator(100))

fig.savefig(r"F:\Insert_Path_to_Profile_Number_Season.png", bbox_inches="tight", pad_inches=0.25, dpi=1200) # Update this file path

In [ ]:
##########################################

In [ ]:
## Get Profile Average Temperatures -- Figure 3B
EntireProf = pd.read_csv(r"Insert_File_Path_for_All_Entire_Profile_Averages.csv") # Created for FIg 4B ## Update this file path
Prof_Avg = pd.merge(KW_Prof_Results_Riv20, EntireProf, on=['Month', 'Day','Year','Assgn_dam'], how='inner')

# Assign the season 
Prof_Avg["Season"] = np.nan
Prof_Avg["Season"] = np.where(Prof_Avg['Month'].isin([12, 1, 2]), 'Winter', Prof_Avg['Season'])
Prof_Avg["Season"] = np.where(Prof_Avg['Month'].isin([3, 4, 5]), 'Spring', Prof_Avg['Season'])
Prof_Avg["Season"] = np.where(Prof_Avg['Month'].isin([6, 7, 8]), 'Summer', Prof_Avg['Season'])
Prof_Avg["Season"] = np.where(Prof_Avg['Month'].isin([9, 10, 11]), 'Fall', Prof_Avg['Season'])

# Plot the Profile Average Temperatures
ColorPalette = ['#031730','#F94994', '#006A5C', '#EB9911']
HueOrder = ["Winter","Spring","Summer","Fall"]

fig, ax=plt.subplots(figsize=(6,4))
ax = sns.boxenplot(data=Prof_Avg, y="Prof_Avg", hue = "Season", hue_order = HueOrder, palette = ColorPalette, width = 0.75)
ax.set_ylabel( "Profile Average Temperature (°C)", fontsize=14) 
ax.set_title("Average Profile Temperature Distribution", fontsize = 14)
ax.tick_params(labelsize=12)
ax.yaxis.set_minor_locator(MultipleLocator(5))
ax.legend(title=None, fontsize=12)

fig.savefig(r"F:\Insert_File_Path_for_Profile_AvgTemps_Season.png", bbox_inches="tight", pad_inches=0.25, dpi=1200) # Update this file path

In [ ]:
#################################################
## Investigate the Downstream Differences by Dam Type ##
################################################

In [ ]:
## Get the temperature differences by dam type (with a reservoir or without a reservoir ) and direction of change --- Figure 3C ###
# Prep the data
Dam_Info = Dams[['grod_id','type','dataset']]
Dam_Info['dataset'].unique()

# Make an easier code for hydropower variables -- Simplified options for analysis
Dam_Info['Resv']  = np.nan
Dam_Info['Resv'] = np.where((Dam_Info['dataset'] == 'Hydropower dam associated with reservoir and power plant') , 'Reservoir', Dam_Info['Resv'])
Dam_Info['Resv'] = np.where((Dam_Info['dataset'] == 'Hydropower dam associated with reservoir; no power plant') , 'Reservoir', Dam_Info['Resv'])
Dam_Info['Resv'] = np.where((Dam_Info['dataset'] == 'Hydropower dam associated with power plant; no reservoir') , 'No Reservoir', Dam_Info['Resv'])
Dam_Info['Resv'] = np.where((Dam_Info['dataset'] == 'Hydropower dam only; no reservoir or power plant') , 'No Reservoir', Dam_Info['Resv'])

## Assessed the dams w/o Hydropower manually for reservoirs (e.g., imagery, NID, SWORD node classification) ##
# Pull in the list of dams and classifications 
NonHydroTypes = pd.read_csv(r"F:\Insert_Path_for_Dam_Reservoir_Check.csv") # Update this file path

## Join the Data & Get the Dam Counts ##
DamTypeRes = pd.merge(Dam_Info, NonHydroTypes, on="grod_id", how="left")
DamTypeRes["Resv"] = np.where((DamTypeRes['Manual_Check'] == 'NoReserv') , 'No Reservoir', DamTypeRes['Resv'])
DamTypeRes["Resv"] = np.where((DamTypeRes['Manual_Check'] == 'Reserv') , 'Reservoir', DamTypeRes['Resv'])

# Join to get the Dam Totals
Dam_Totals = pd.merge(KW_Prof_Results_Riv20[['Assgn_dam','Month','Day','Year','KW_Sig_05']], DamTypeRes, left_on = 'Assgn_dam', right_on = 'grod_id' , how='left').drop(columns = [ 'grod_id'])

# Get the differences
Diff_Sign = Profile_Avgs_Riv[:]
Diff_Sign["Diff_Direction"] = np.nan
Diff_Sign["Diff_Direction"] = np.where((Diff_Sign["Temp_Diff"] < 0) , 'Cooler', Diff_Sign["Diff_Direction"])
Diff_Sign["Diff_Direction"] = np.where((Diff_Sign["Temp_Diff"] > 0) , 'Warmer', Diff_Sign["Diff_Direction"])

# Join Temp Directions with the Dam Info
Dam_Totals_Diff = pd.merge(Dam_Totals, Diff_Sign[['Month', 'Day','Year','Assgn_dam','Temp_Diff', 'Diff_Direction']], on=['Month', 'Day','Year','Assgn_dam'], how='left')
Dam_Totals_Diff["Diff_Direction"]= np.where((Dam_Totals_Diff["Diff_Direction"].isna()) , 'No Change', Dam_Totals_Diff["Diff_Direction"]) # can't have nulls in a later step

# Get the Profile Counts by Dam 
DamType_Info_ResSplit = Dam_Totals_Diff.groupby(["Assgn_dam","type","Resv","KW_Sig_05","Diff_Direction"]).agg({'Month': ['count']})
DamType_Info_ResSplit.columns = ["Profile Count"]
DamType_Info_ResSplit = DamType_Info_ResSplit.reset_index()

# Get Dam Type Profile Stats -- Supplemental Table 1
# Get the Data Range
def calculate_range(x):
    return x.max() - x.min()

ResNoRes_Details  = Dam_Totals_Diff[:]
ResNoRes_Details['Temp_Diff'] = np.where(ResNoRes_Details['Temp_Diff'].isna(), 0, ResNoRes_Details['Temp_Diff'])
ResProfCount_Type_Dir = ResNoRes_Details.groupby(["Resv","Diff_Direction"]).agg({'Month': ['count'], 'Temp_Diff': ['mean', 'std', calculate_range]})
ResProfCount_Type_Dir.columns = ['Profile_Count','Avg_Diff', 'STDV', 'Range']
ResProfCount_Type_Dir = ResProfCount_Type_Dir.reset_index()

## Save the Data 
ResProfCount_Type_Dir.to_csv(r"F:\Insert_Path_for_Diffeerence_Stats_by_Dam_Type.csv")  # Update this file path

## Look at just the changes 
ResNoRes = ResProfCount_Type_Dir[ResProfCount_Type_Dir['Diff_Direction']!= 'No Change']

# Preview the Data -- Table S2
ResNoRes

In [ ]:
### Format the Data for Rose Plot, Plot, &  Save ###
Profile_PolarPlot = ResProfCount_Type_Dir[:]

# Name the groups 
Profile_PolarPlot["GroupName"] = np.nan
# Reservoir
Profile_PolarPlot["GroupName"] = np.where(((Profile_PolarPlot['Diff_Direction'] == 'Cooler') & (Profile_PolarPlot["Resv"] == "Reservoir")), 'Reservoir-Cooler', Profile_PolarPlot['GroupName'])
Profile_PolarPlot["GroupName"] = np.where(((Profile_PolarPlot['Diff_Direction'] == 'Warmer') & (Profile_PolarPlot["Resv"] == "Reservoir")), 'Reservoir-Warmer', Profile_PolarPlot['GroupName'])
Profile_PolarPlot["GroupName"] = np.where(((Profile_PolarPlot['Diff_Direction'] == 'No Change') & (Profile_PolarPlot["Resv"] == "Reservoir")), 'Reservoir-NC', Profile_PolarPlot['GroupName'])
# No Reservoir
Profile_PolarPlot["GroupName"] = np.where(((Profile_PolarPlot['Diff_Direction'] == 'Cooler') & (Profile_PolarPlot["Resv"] == "No Reservoir")), 'No Reservoir-Cooler', Profile_PolarPlot['GroupName'])
Profile_PolarPlot["GroupName"] = np.where(((Profile_PolarPlot['Diff_Direction'] == 'Warmer') & (Profile_PolarPlot["Resv"] == "No Reservoir")), 'No Reservoir-Warmer', Profile_PolarPlot['GroupName'])
Profile_PolarPlot["GroupName"] = np.where(((Profile_PolarPlot['Diff_Direction'] == 'No Change') & (Profile_PolarPlot["Resv"] == "No Reservoir")), 'No Reservoir-NC', Profile_PolarPlot['GroupName'])

import plotly.express as px
fig = px.bar_polar(Profile_PolarPlot, r="Profile_Count", theta="GroupName", color="Avg_Diff", color_continuous_scale = ["#1c7379", "#eeddd4", "#c64436"],).update_layout(polar_hole=0.1, height=600, width=800, margin=dict(b=30, t=30, l=0, r=0))
fig.show()

fig.write_image(r'F:\Insert_Path_for_ProfileStats_ReservType.pdf')  # Update this file path

In [ ]:
###################################################################

In [ ]:
### Plot the spread of the differences by dam type and direction ### --- Figure S2

# Get the siginifcant profiles 
SignifProf_DamType = ResNoRes_Details[ResNoRes_Details['Diff_Direction']!= 'No Change' ]

# Set the plot details 
ColorPalette = ["#1c7379", "#c64436"]
HueOrder = ["Cooler","Warmer"]

# Plot
fig, ax=plt.subplots(figsize=(3,5))
ax = sns.boxenplot(data=SignifProf_DamType, x='Resv', y='Temp_Diff', hue = "Diff_Direction", hue_order = HueOrder, palette = ColorPalette, linecolor= "#1F201F", width = 0.75, flier_kws=dict(facecolor="#1F201F", linewidth=.25))
ax.set_xlabel( "") 
ax.set_ylabel( "Downstream Temperature Difference (°C)", fontsize=12) 
ax.tick_params(labelsize=10)
ax.legend(title=None, loc = 'lower right' ,fontsize=10)
ax.set_ylim(-15,10)

# Save Figure
fig.savefig(r"F:\Insert_Path_for_Signif_DS_TempDiffs.pdf", bbox_inches="tight", pad_inches=0.25, dpi=300) # Update this file path


In [ ]:
#######################################################################

In [ ]:
### Show Example Longitudinal Profiles (Cooler, Warmer, No Change) ### --- Figure 1C

## Plots will need the UP and DS averages for each profile  ##
# Get the Profiles and Nodes -- Copy
Signif_Prof_Points = Signif_Riv20_Profile_Nodes[Signif_Riv20_Profile_Nodes["KW_Sig_05"] == "Significant"]

# Of The Significant Profiles -- Avg Diff Up Vs Ds 
Signif_Prof_Points_UP = Signif_Prof_Points[(Signif_Prof_Points['Up_Ds_Grp'] == "Upstream River")]
Signif_Prof_Points_UP_avg = Signif_Prof_Points_UP.groupby(['Month','Day','Year','Assgn_dam']).agg({'Avg_Temp': ['mean']})
Signif_Prof_Points_UP_avg.columns = ['Up_Avg']
Signif_Prof_Points_UP_avg = Signif_Prof_Points_UP_avg.reset_index()

Signif_Prof_Points_DS = Signif_Prof_Points[(Signif_Prof_Points['Up_Ds_Grp'] == 'Downstream River')]
Signif_Prof_Points_DS_avg = Signif_Prof_Points_DS.groupby(['Month','Day','Year','Assgn_dam']).agg({'Avg_Temp': ['mean']})
Signif_Prof_Points_DS_avg.columns = ['Ds_Avg']
Signif_Prof_Points_DS_avg = Signif_Prof_Points_DS_avg.reset_index()

UpDS_Avg = pd.merge(Signif_Prof_Points_UP_avg, Signif_Prof_Points_DS_avg, on=['Month','Day', 'Year','Assgn_dam'], how='outer' )

UpDS_Avg['Temp_Diff'] = UpDS_Avg['Ds_Avg'] - UpDS_Avg['Up_Avg'] 

## Colder Example  ## 
# Set up the plot 
fig, ax=plt.subplots(figsize=(8,6))

# Get the profile for Dam 306 on Sept. 8, 2021
ProfTest = Signif_Prof_Points[(Signif_Prof_Points['Month'] == 9) & (Signif_Prof_Points['Day'] == 8) & (Signif_Prof_Points['Year'] == 2021) & (Signif_Prof_Points['Assgn_dam'] == 306)]
ProfTest_Avg_up = Profile_Avgs_Riv[(Profile_Avgs_Riv['Month'] == 9) & (Profile_Avgs_Riv['Day'] == 8) & (Profile_Avgs_Riv['Year'] == 2021)& (Profile_Avgs_Riv['Assgn_dam'] == 306)].Up_Avg
ProfTest_Avg_ds = Profile_Avgs_Riv[(Profile_Avgs_Riv['Month'] == 9) & (Profile_Avgs_Riv['Day'] == 8) & (Profile_Avgs_Riv['Year'] == 2021) &(Profile_Avgs_Riv['Assgn_dam'] == 306)].Ds_Avg

# Plot 
sns.lineplot(data = ProfTest[(ProfTest['Up_Ds_Grp'] =="Upstream River")], x = 'Dam_Dist_km', y ='Avg_Temp', color = "#18A790", linewidth=2, label = "Upstream River")
sns.lineplot(data = ProfTest[(ProfTest['Up_Ds_Grp'] =="Upstream Reservoir")], x = 'Dam_Dist_km', y ='Avg_Temp', color = "#1877A8", linewidth=2, label = "Reservoir")
sns.lineplot(data = ProfTest[(ProfTest['Up_Ds_Grp'] =="Downstream River")], x = 'Dam_Dist_km', y ='Avg_Temp', color = "#1C4BC4", label = "Downstream River")
plt.hlines(y=ProfTest_Avg_up, xmin=(ProfTest[ProfTest['Up_Ds_Grp'] =="Upstream River"].Dam_Dist_km.min()), xmax= (ProfTest[ProfTest['Up_Ds_Grp'] =="Upstream River"].Dam_Dist_km.max()) , linewidth=2, color='#7F0824', alpha = 0.8)
plt.hlines(y=ProfTest_Avg_ds, xmin= (ProfTest[ProfTest["Up_Ds"]=="Downstream"].Dam_Dist_km.min()), xmax= (ProfTest[ProfTest["Up_Ds"]=="Downstream"].Dam_Dist_km.max()), linewidth=3, color='#7F0824',alpha = 0.8, label = "Average Temperature")
plt.axvline(x=0.15, linewidth=3, color='#fb5607', linestyle='--', zorder = 1, label = 'Dam')
plt.xlabel("Distance Downstream (km)", fontsize = 16)
plt.xticks(fontsize=16)
plt.xlim(-45,21)
plt.ylabel("Water Surface Temperature (°C)", fontsize = 16)
plt.yticks(fontsize=12)
plt.legend(loc='lower left', prop={'size': 16})
plt.locator_params(axis='y', nbins= 6)
plt.tight_layout()

fig.savefig(r"F:\Insert_Path_for_Colder_Profile_09082021.png", bbox_inches="tight", pad_inches=0.25, dpi=1200) # Update this file path

In [ ]:
## Warmer Example ## 
# Set up plot
fig, ax=plt.subplots(figsize=(8,6))

# Get profile for Dam 306 on Feb. 15, 2021
ProfTest = Signif_Prof_Points[(Signif_Prof_Points['Month'] == 2) & (Signif_Prof_Points['Day'] == 15) &(Signif_Prof_Points['Assgn_dam'] == 306) ]
ProfTest_Avg_up = Profile_Avgs_Riv[(Profile_Avgs_Riv['Month'] == 2) & (Profile_Avgs_Riv['Day'] == 15) &(Profile_Avgs_Riv['Assgn_dam'] == 306)].Up_Avg
ProfTest_Avg_ds = Profile_Avgs_Riv[(Profile_Avgs_Riv['Month'] == 2) & (Profile_Avgs_Riv['Day'] == 15) &(Profile_Avgs_Riv['Assgn_dam'] == 306)].Ds_Avg

# Single spurious point, removed for visualization
MisclassPoint = ProfTest[(ProfTest['Up_Ds_Grp'] =="Upstream Reservoir")].Dam_Dist_km.min()

# Plot
sns.lineplot(data = ProfTest[(ProfTest['Up_Ds_Grp'] =="Upstream River") & (ProfTest['Dam_Dist_km'] <= MisclassPoint)], x = 'Dam_Dist_km', y ='Avg_Temp',  linewidth= 2, color = "#18A790", label = "Upstream River")
sns.lineplot(data = ProfTest[(ProfTest['Up_Ds_Grp'] =="Upstream Reservoir")], x = 'Dam_Dist_km', y ='Avg_Temp', linewidth= 2, color = "#1877A8", label = "Reservoir")
sns.lineplot(data = ProfTest[(ProfTest['Up_Ds_Grp'] =="Downstream River")], x = 'Dam_Dist_km', y ='Avg_Temp', linewidth= 2, color = "#1C4BC4", label = "Downstream River")
plt.xlim(-45,21)
plt.hlines(y=ProfTest_Avg_up, xmin=(ProfTest[ProfTest['Up_Ds_Grp'] =="Upstream River"].Dam_Dist_km.min()), xmax=MisclassPoint , linewidth=3, color='#7F0824', alpha = 0.8)
plt.hlines(y=ProfTest_Avg_ds, xmin= 0, xmax= (ProfTest[ProfTest["Up_Ds_Grp"]=="Downstream River"].Dam_Dist_km.max()), linewidth=3, color='#7F0824',alpha = 0.8, label = "Average Temperature")
plt.axvline(x=0.1, linewidth=3, color='#fb5607', linestyle='--', zorder = 1, label = 'Dam')
plt.xlabel("Distance Downstream (km)", fontsize = 16)
plt.xticks(fontsize=14)
plt.ylabel("Water Surface Temperature (°C)", fontsize = 16)
plt.yticks(fontsize=14)
plt.legend([], [], frameon=False)
plt.locator_params(axis='y', nbins= 6)
plt.tight_layout()
fig.savefig(r"F:\Insert_Path_for_Warmer_Profile_02152022.png", bbox_inches="tight", pad_inches=0.25, dpi=1200) # Update this file path

In [ ]:
#### No Change Example ####
# Set up the plot
fig, ax=plt.subplots(figsize=(8,6))

#  Get profile for Dam 306 on Dec. 21, 2024
ProfTest = NS_Riv20_Profile_Nodes[(NS_Riv20_Profile_Nodes['Month'] == 12) & (NS_Riv20_Profile_Nodes['Day'] == 21) & (NS_Riv20_Profile_Nodes['Year'] == 2024) & (NS_Riv20_Profile_Nodes['Assgn_dam'] == 306)]
ProfTest_Avg_up = ProfTest[ProfTest['Up_Ds_Grp'] == "Upstream River"].Avg_Temp.mean()
ProfTest_Avg_ds = ProfTest[ProfTest['Up_Ds_Grp'] == "Downstream River"].Avg_Temp.mean()

# Single spurious point, removed for visualization
MisclassPoint = ProfTest[(ProfTest['Up_Ds_Grp'] =="Upstream Reservoir")].Dam_Dist_km.min()

# Plot
sns.lineplot(data = ProfTest[(ProfTest['Up_Ds_Grp'] =="Upstream River") & (ProfTest['Dam_Dist_km'] <= MisclassPoint)], x = 'Dam_Dist_km', y ='Avg_Temp',  linewidth= 2, color = "#18A790", label = "Upstream River")
sns.lineplot(data = ProfTest[(ProfTest['Up_Ds_Grp'] =="Upstream Reservoir")], x = 'Dam_Dist_km', y ='Avg_Temp', linewidth= 2, color = "#1877A8", label = "Reservoir")
sns.lineplot(data = ProfTest[(ProfTest['Up_Ds_Grp'] =="Downstream River")], x = 'Dam_Dist_km', y ='Avg_Temp', linewidth= 2, color = "#1C4BC4", label = "Downstream River")
plt.xlim(-45,21)
plt.hlines(y=ProfTest_Avg_up, xmin=(ProfTest[ProfTest['Up_Ds_Grp'] =="Upstream River"].Dam_Dist_km.min()), xmax=MisclassPoint , linewidth=3, color='#7F0824', alpha = 0.8)
plt.hlines(y=ProfTest_Avg_ds, xmin= 0, xmax= (ProfTest[ProfTest["Up_Ds_Grp"]=="Downstream River"].Dam_Dist_km.max()), linewidth=3, color='#7F0824',alpha = 0.8, label = "Average Temperature")
plt.axvline(x=0.1, linewidth=3, color='#fb5607', linestyle='--', zorder = 1, label = 'Dam')
plt.xlabel("Distance Downstream (km)", fontsize = 16)
plt.xticks(fontsize=14)
plt.ylabel("Water Surface Temperature (°C)", fontsize = 16)
plt.yticks(fontsize=14)
plt.legend([], [], frameon=False)
plt.locator_params(axis='y', nbins= 6)
plt.tight_layout()

fig.savefig(r"F:\Insert_Path_for_NoChange_Profile_12212024.png", bbox_inches="tight", pad_inches=0.25, dpi=1200)# Update this file path


In [ ]:
#####################
## Extra Profile Details ##
#####################

In [ ]:
## Get the some Upstream Counts ##
print("Median Upstream River length: " + str(round(((UpAnDown.NodeCount_y.median() * 200)/1000))))
print("Mean Upstream River length: " + str(round(((UpAnDown.NodeCount_y.mean() * 200)/1000))))

In [ ]:
#########################################################
## Compare the Observed Widths to Accuracy Assessment Widths ##
########################################################

In [ ]:
##### Accuracy Assessment Comparision Graphics 
## Get the widths of nodes used in the analysis
Profile_Nodes_widths = Profile_Differences[Profile_Differences['Dam_Dist_km'] <= 60]

# Get Node Width Stats 
print("Median Landsat Observation Width: ", str(round(Profile_Nodes_widths["Avg_RWC_Wid"].median())))

# Pull in the widths at the gage locations 
AANodeWidths = pd.read_csv(r"F:\Insert_Path_for_Widths_AA.csv") # Update this file path #Created during Accuracy Assessment
JustAAWidths_df = pd.concat([AANodeWidths['Avg_RWC_Wid_x'], AANodeWidths['Avg_RWC_Wid_y']]).to_frame()
JustAAWidths_df.columns = ['Width']
JustAAWidths_df["NodeType"] = "USGS Gage Width"
print("Median Gage Observation Width: ", str(round(JustAAWidths_df["Width"].median())))

## Observation Widths
Obvs_Widths = Profile_Nodes_widths[["Avg_RWC_Wid"]]
Obvs_Widths = Obvs_Widths.rename(columns={"Avg_RWC_Wid": "Width"})
Obvs_Widths["NodeType"] = "Profile Node Width"

# Combine 
AllNodeWidths = pd.concat([JustAAWidths_df, Obvs_Widths], ignore_index=True)

In [ ]:
### Plot the Widths 
# Set the plot details
my_palette = ['#C1B3E3', '#fb5012']

fig, ax=plt.subplots(figsize=(6,2))
ax = sns.boxplot(data=AllNodeWidths, x="Width", hue ="NodeType", linecolor="#1A0E00", linewidth=1.5, width=0.75, gap=0.25,  palette=my_palette, showfliers = False) # Comment this out to see outliers
ax.set_xlim(75, 1000)
ax.set_xlabel( "River Width (m)", fontsize=14) 
ax.set_title("Distribution of Observation Widths", fontsize = 14)
ax.tick_params(labelsize=12)
ax.xaxis.set_minor_locator(MultipleLocator(100))
ax.legend().set_title('')
fig.savefig(r"F:\Insert_Path_for_All_Node_Widths.png", bbox_inches="tight", pad_inches=0.25, dpi=1200) # Update this file path